# Assignment 2

## 1. Download data

In [3]:
## Install
!conda install -c conda-forge folium=0.5.0 --yes
!pip install lxml
!pip install html5lib
!pip install requests
!pip install beautifulsoup4

import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner

from bs4 import BeautifulSoup


Solving environment: done

# All requested packages already installed.



In [4]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
soup = BeautifulSoup(requests.get(url).text, 'lxml')
table = soup.find("table")
table_rows = table.tbody.find_all("tr")
data = []
for tr in table_rows:
    row = [tr.text for tr in tr.find_all("td")]
    if row != [] and row[1] != "Not assigned":
        if "Not assigned" in row[2]: 
            row[2] = row[1]
        data.append(row)

## 2. Pre-process data

In [5]:
# Replace the /n and also skip the 'Not assigned' values, then reindex
df = pd.DataFrame(data, columns = ["PostalCode", "Borough", "Neighborhood"])
df["Neighborhood"] = df["Neighborhood"].str.replace("\n","")
df["PostalCode"] = df["PostalCode"].str.replace("\n","")
df["Borough"] = df["Borough"].str.replace("\n","")
df.drop(df[df["Borough"] == "Not assigned"].index, axis = 0, inplace = True)
df.reset_index(drop = True, inplace = True)
df.head(5)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


## 3. Print the count

In [6]:
print("Number of rows in dataframe: ", df.shape[0])

Number of rows in dataframe:  103


# 4. Merge data to get the Latitude and Longitude

In [8]:
# Fetch the geospatial table containg the lat and long data
geospatial = pd.read_csv("https://cocl.us/Geospatial_data")
geospatial.rename(columns = {"Postal Code": "Postal code"}, inplace = True)
geospatial.head()

,Postal code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [9]:
merged_df = pd.merge(df, geospatial, how='left', left_on = 'PostalCode', right_on = 'Postal code')
merged_df.head(15)

,PostalCode,Borough,Neighborhood,Postal code,Latitude,Longitude
0,M3A,North York,Parkwoods,M3A,43.753259,-79.329656
1,M4A,North York,Victoria Village,M4A,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",M5A,43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",M6A,43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",M7A,43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",M9A,43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",M1B,43.806686,-79.194353
7,M3B,North York,Don Mills,M3B,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",M4B,43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",M5B,43.657162,-79.378937
